<a href="https://colab.research.google.com/github/ufda/RL-FX-trading/blob/main/RL_FX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stable_baselines3

     |████████████████████████████████| 172 kB 4.1 MB/s 


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



###**Execute following code to prevent Random Disconnects**


In [ ]:
import IPython
display(IPython.display.Javascript('''
  function reconnect()
  {
    console.log("Script Working");
    btn = document.querySelector("colab-connect-button");
    if (btn != null)
    {
      console.log("Working Clicking Reconnect Button 1");
      btn.click();
    }
    btn = document.getElementById('ok');
    if (btn != null)
    {
      console.log("Working Clicking Reconnect Button 1");
      btn.click();
    }
  }
  setInterval(reconnect, 5000);
'''))

print ("Finished")

<IPython.core.display.Javascript object>

Finished


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
%cd "drive"
%cd "My Drive"
%cd "RL_FX"

/content/drive
/content/drive/My Drive
/content/drive/My Drive/RL_FX


In [ ]:
!git clone https://github.com/ufda/RL-FX-trading
#%cd DFL-Colab-Shraeyas
#!pip install -r requirements-colab.txt
#!echo "Done."

Cloning into 'RL-FX-trading'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 88 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [ ]:
%cd "data"
%ls


/content/drive/My Drive/RL_FX/data
episode-0-trained-agent.h5  episode-1-trained-agent.h5  EU_M5_2017_2020.csv


In [ ]:
import pandas as pd
df = pd.read_csv("EU_M5_2017_2020.csv")

In [ ]:
s =int(len(df)*0.7)
df1 =df[:s]
df2 = df[s:]
df1

,Gmt time,Open,High,Low,Close,Volume
0,15.12.2017 00:00:00.000,1.17745,1.17764,1.17732,1.17749,674.72
1,15.12.2017 00:05:00.000,1.17749,1.17765,1.17739,1.17760,478.71
2,15.12.2017 00:10:00.000,1.17760,1.17783,1.17759,1.17783,478.75
3,15.12.2017 00:15:00.000,1.17784,1.17804,1.17779,1.17784,396.89
4,15.12.2017 00:20:00.000,1.17789,1.17804,1.17772,1.17787,498.75
...,...,...,...,...,...,...
156442,22.01.2020 01:20:00.000,1.10807,1.10812,1.10804,1.10806,184.19
156443,22.01.2020 01:25:00.000,1.10806,1.10808,1.10802,1.10802,147.53
156444,22.01.2020 01:30:00.000,1.10802,1.10802,1.10776,1.10802,376.86
156445,22.01.2020 01:35:00.000,1.10802,1.10819,1.10802,1.10819,239.66


In [ ]:
df2

,Gmt time,Open,High,Low,Close,Volume
156447,22.01.2020 01:45:00.000,1.10807,1.10823,1.10802,1.10813,251.29
156448,22.01.2020 01:50:00.000,1.10814,1.10818,1.10807,1.10807,362.21
156449,22.01.2020 01:55:00.000,1.10808,1.10818,1.10805,1.10818,212.62
156450,22.01.2020 02:00:00.000,1.10818,1.10818,1.10811,1.10818,221.72
156451,22.01.2020 02:05:00.000,1.10817,1.10823,1.10794,1.10797,298.62
...,...,...,...,...,...,...
223492,11.12.2020 21:35:00.000,1.21151,1.21153,1.21143,1.21145,101.22
223493,11.12.2020 21:40:00.000,1.21145,1.21151,1.21142,1.21145,189.38
223494,11.12.2020 21:45:00.000,1.21148,1.21159,1.21113,1.21142,1439.78
223495,11.12.2020 21:50:00.000,1.21143,1.21145,1.21104,1.21125,3560.32


# RL Agent

In [ ]:
import tensorflow as tf
import numpy as np
import random
from collections import deque

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)


class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size
        self.action_size = 3  # sit, long, short
        self.memory = deque(maxlen=1000)
        self.in_trade = False
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model_name = model_name
        if is_eval:
            tf.keras.models.load_model(model_name)
        else:
            self.model = self.build_actor()

    def build_actor(self):

        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.LSTM(512, input_shape=(self.state_size, 4)))
        model.add(tf.keras.layers.Dense(self.action_size, activation='linear'))
        opt = tf.keras.optimizers.Adam(learning_rate=0.000001)
        model.compile(loss='mse', optimizer=opt)

        return model

    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            return random.randrange(self.action_size)

        state = state.reshape(1, self.state_size, 4)
        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batchsize):
        mini_batch = random.sample(self.memory, batchsize)

        for state, action, reward, next_state, done in mini_batch:
            next_state = next_state.reshape(1, self.state_size, 4)
            state = state.reshape(1, self.state_size, 4)

            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            else:
                target = reward

            target_f = self.model.predict(state)
            
            target_f[0][action] = target
            
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


# RL Train

In [ ]:
import numpy as np
from tqdm import tqdm
import pandas as pd

def getState(data, start, window):
    result = data[start:start + window]
    return np.array(result)


def evalTrade(a, tr, next_step):
    if tr[0] == 1:  # LONG
        if next_step[-1][1] >= tr[3]:  # TP
            r = 1
            a.in_trade = False
            return [1, r]

        elif next_step[-1][3] <= tr[2]:  # SL
            r = -1
            a.in_trade = False
            return [1, r]

    elif tr[0] == 2:  # SHORT
        if next_step[-1][3] <= tr[3]:  # TP
            r = 1
            a.in_trade = False
            return [2, r]

        elif next_step[-1][1] >= tr[2]:  # SL
            r = -1
            a.in_trade = False
            return [2, r]
    return [0, 0]

def evalPass(current, next):
    # Reward pass if action prevented SL
    long = takeTrade(1, current[-1])
    short = takeTrade(2, current[-1])
    if long[2] >= next[-1][3] or short[2] <= next[-1][1]:
        return 0.1
    return 0


def invScale(data, minimum, maximum):
    return data * (maximum - minimum) + minimum


def takeTrade(action, ohlc):
    unscaled_close = invScale(ohlc, data_min, data_max)[3]
    close = ohlc[3]

    if action == 1:  # (long/short, entry, SL, TP)
        sl = (unscaled_close - 0.0010 - data_min) / (data_max - data_min)
        tp = (unscaled_close + 0.0030 - data_min) / (data_max - data_min)
        return action, close, sl, tp

    elif action == 2:
        sl = (unscaled_close + 0.0010 - data_min) / (data_max - data_min)
        tp = (unscaled_close - 0.0030 - data_min) / (data_max - data_min)
        return action, close, sl, tp


df = pd.read_csv('EU_M5_2017_2020.csv',
                 index_col='Local time',
                 parse_dates=True,
                 skiprows=1,
                 nrows=5000,
                 names=['Local time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df = df.drop('Volume', 1)

s = int(len(df))
df = df[:s]

data_max = df['High'].max()
data_min = df['Low'].min()
df = np.array(df)
df = (df - data_min) / (data_max - data_min)

MIN_MEM_SIZE = 128
BATCH_SIZE = 64
WINDOW_SIZE = 90
EPISODE_LENGTH = len(df) - WINDOW_SIZE
ACTIONS = ['PASS', 'LONG', 'SHORT']

agent = Agent(WINDOW_SIZE)
win, loss = 0, 0

loop = tqdm(total=EPISODE_LENGTH, position=0, leave=False)
for ep in range(10):

    print('starting ep ', ep)
    ret = 0

    for t in range(EPISODE_LENGTH):
        loop.set_description('Training ...'.format(t))
        loop.update(1)

        done = False
        if t == 0:
            state = getState(df, t, WINDOW_SIZE)  # (90, 4) size

        close = state[-1][3]
        reward = 0
        if not agent.in_trade:
            action = agent.act(state)

            if action == 1:
                trade = takeTrade(action, state[-1])
                entry_state = state
                entry_t = t
                agent.in_trade = True
                print(f'--t:{t} OPEN {ACTIONS[action]} --')

            elif action == 2:
                trade = takeTrade(action, state[-1])
                entry_state = state
                entry_t = t
                agent.in_trade = True
                print(f'--t:{t} OPEN {ACTIONS[action]} --')

        next_state = getState(df, t + 1, WINDOW_SIZE)

        if agent.in_trade:
            result = evalTrade(agent, trade, next_state)
            if result[1] != 0:
                if result[1] >= 0:
                    win += 1
                    ret += 3
                else:
                    loss += 1
                    ret -= 1
                reward += result[1]
                agent.memory.append((entry_state, action, reward, next_state, done))

                #print(f'--t:{t + 1} CLOSING {ACTIONS[result[0]]} ({reward})')

                if len(agent.memory) >= MIN_MEM_SIZE:
                    print('--- Fitting new Weights ---:')
                    agent.expReplay(BATCH_SIZE)

        # Passing
        else:
            reward = evalPass(state, next_state)
            print(f'--t:{t} NO TRADE ({reward})')
            agent.memory.append((state, action, reward, next_state, done))
            if len(agent.memory) >= MIN_MEM_SIZE:
                print('--- Fitting new Weights ---')
                agent.expReplay(BATCH_SIZE)

        if t == EPISODE_LENGTH - 1 or ret <= -100:
            done = True
            agent.memory.append((state, action, reward, next_state, done))
            state = next_state

            print(f"--- Cumulative return Episode {ep}: {ret} ---")
            if win + loss > 0:
                print(f"--- Winrate: {win / (win + loss)} ---")
            break

        state = next_state

    agent.model.save(f'episode-{ep}-trained-agent.h5')


FileNotFoundError: ignored